# APT model: Famma-Macbeth Regression

In [1]:
from rqdata_utils import *
import pandas
import numpy as np
import scipy as sp
import alphalens as al
%matplotlib inline

## Loading Data

In [2]:
price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv","cn_instrument_info_2012_2018.csv","cn_equity_daily_2012_2018.csv","sectorCode")

In [3]:
equity_df.head()

return   close  total_turnover      volume  \
date       order_book_id                                                 
2012-01-04 000001.XSHE   -0.027582  5.1224    2.275637e+08  40894428.0   
           000002.XSHE   -0.018742  6.0525    3.559891e+08  47432958.0   
           000004.XSHE   -0.022250  7.9100    3.763833e+06    465469.0   
           000005.XSHE    0.000000  3.8600    0.000000e+00         0.0   
           000006.XSHE   -0.009756  2.6766    7.619286e+06   2513811.0   

                            week   month report_quarter    market_cap  \
date       order_book_id                                                
2012-01-04 000001.XSHE    0.5775  0.4331            NaN           NaN   
           000002.XSHE    0.3711  0.4030         2011q3  8.059489e+10   
           000004.XSHE    0.5720  0.7506         2011q3  6.642556e+08   
           000005.XSHE    0.0000  0.0000         2011q3  3.529328e+09   
           000006.XSHE    0.1416  0.1667         2011q3  4.015370e+09   

                          a_share_market_val_2  \
date       order_book_id                         
2012-01-04 000001.XSHE                     NaN   
           000002.XSHE            7.082120e+10   
           000004.XSHE            6.634549e+08   
           000005.XSHE            3.527048e+09   
           000006.XSHE            3.929464e+09   

                          cash_received_from_sales_of_goods  pb_ratio  \
date       order_book_id                                                
2012-01-04 000001.XSHE                                  NaN       NaN   
           000002.XSHE                         7.516785e+10    1.5216   
           000004.XSHE                         5.949968e+07    8.8175   
           000005.XSHE                         2.565851e+07    5.3480   
           000006.XSHE                         2.531436e+09    1.4348   

                            net_profit  ps_ratio   sectorCode  
date       order_book_id                                       
2012-01-04 000001.XSHE             NaN       NaN   Financials  
           000002.XSHE    4.106349e+09    0.8679   Financials  
           000004.XSHE    4.500363e+06   37.5796   HealthCare  
           000005.XSHE    1.365665e+07 -347.2191  Industrials  
           000006.XSHE    2.763917e+08    1.4139   Financials

In [4]:
healthcareUniverse = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values
len(healthcareUniverse)

164

In [5]:
def equity_universe_filtering(equity_df, universe):
    universeFilter = [book_id in set(universe) for book_id in equity_df.index.get_level_values(level=1).values]
    return equity_df[universeFilter]

In [6]:
healthcare_equity_df = equity_universe_filtering(equity_df, healthcareUniverse)
healthcare_equity_df.head()

return    close  total_turnover     volume  \
date       order_book_id                                                 
2012-01-04 000004.XSHE   -0.022250   7.9100      3763832.88   465469.0   
           000028.XSHE   -0.045433  19.8422      9326924.28   450553.0   
           000150.XSHE   -0.030295   3.1737      3109304.50   952600.0   
           000153.XSHE   -0.028053   5.7700      9673054.49  1596020.0   
           000403.XSHE    0.000000   3.1625            0.00        0.0   

                            week   month report_quarter    market_cap  \
date       order_book_id                                                
2012-01-04 000004.XSHE    0.5720  0.7506         2011q3  6.642556e+08   
           000028.XSHE    0.4201  0.2722         2011q3  5.872485e+09   
           000150.XSHE    0.3460  0.3610         2011q3  1.036800e+09   
           000153.XSHE    0.6830  2.4594         2011q3  1.531454e+09   
           000403.XSHE    0.0000  0.0000            NaN           NaN   

                          a_share_market_val_2  \
date       order_book_id                         
2012-01-04 000004.XSHE            6.634549e+08   
           000028.XSHE            4.753820e+09   
           000150.XSHE            1.036800e+09   
           000153.XSHE            1.360856e+09   
           000403.XSHE                     NaN   

                          cash_received_from_sales_of_goods  pb_ratio  \
date       order_book_id                                                
2012-01-04 000004.XSHE                         5.949968e+07    8.8175   
           000028.XSHE                         1.053298e+10    4.3493   
           000150.XSHE                         4.913279e+07    1.4763   
           000153.XSHE                         1.329425e+09    2.1169   
           000403.XSHE                                  NaN       NaN   

                            net_profit  ps_ratio  sectorCode  
date       order_book_id                                      
2012-01-04 000004.XSHE    4.500363e+06   37.5796  HealthCare  
           000028.XSHE    2.481834e+08    0.3414  HealthCare  
           000150.XSHE    3.657858e+06    7.8956  HealthCare  
           000153.XSHE    1.560397e+07    0.7818  HealthCare  
           000403.XSHE             NaN       NaN  HealthCare

In [7]:
print("universe ratio: {}%".format(len(healthcare_equity_df)/len(equity_df)*100))

universe ratio: 6.210331877919959%


### benchmark

In [27]:
benchmark_df = pd.read_csv("cn_SH_healthcare_index_2012_2018.csv",names=['date','value'])
benchmark_df = benchmark_df.set_index('date',drop=True)

In [33]:
benchmark_df['return'] = np.log(benchmark_df.shift(1)/benchmark_df).fillna(0)
benchmark_df.head()

,value,return
date,,
2012-01-04,2891.462,0.000000
2012-01-05,2766.955,0.044015
2012-01-06,2744.793,0.008042
2012-01-09,2833.219,-0.031708
2012-01-10,2929.594,-0.033450


## Factor Returns

In [8]:
def equity_factor_return(equity_df, factorColumn, nAllocations, longTop=True):
    equity_copy = equity_df.copy()
#     equity_copy["{}_rank".format(factorColumn)] = equity_copy.groupby(level='date')[factorColumn].rank()
#     equity_copy[equity_copy.groupby(level='date')[factorColumn].nlargest(nAllocations).index]["biggest_{}_{}".format(nAllocations,factorColumn)]=True
    largest = equity_copy[factorColumn].groupby(level='date').nlargest(nAllocations).reset_index(level=0,drop=True)
    smallest = equity_copy[factorColumn].groupby(level='date').nsmallest(nAllocations).reset_index(level=0,drop=True)
    r_largest = equity_copy.loc[largest.index,'return'].groupby(level='date').mean()
    r_smallest = equity_copy.loc[smallest.index,'return'].groupby(level='date').mean()
    LMS = r_largest - r_smallest
    if(longTop):
        return LMS
    else:
        return -LMS

In [9]:
SMB = equity_factor_return(healthcare_equity_df, 'market_cap', 20,longTop=False)
SMB.head()

date
2012-01-04    0.005983
2012-01-05   -0.009098
2012-01-06   -0.004155
2012-01-09    0.014615
2012-01-10    0.006728
Name: return, dtype: float64

In [10]:
HML = equity_factor_return(healthcare_equity_df, 'pb_ratio', 20,longTop=True)
HML.head()

date
2012-01-04    0.005302
2012-01-05   -0.007223
2012-01-06    0.006031
2012-01-09   -0.002597
2012-01-10   -0.010780
Name: return, dtype: float64

In [11]:
import itertools
import statsmodels.api as sm
from statsmodels import regression,stats
import scipy

data = healthcare_equity_df[['return']] # dataframe
data = data.set_index(healthcare_equity_df.index) # elimilate redundant index (whole universe)
asset_list_sizes = [group[1].size for group in data.groupby(level=0)]

# Spreading the factor portfolio data across all assets for each day
SMB_column = [[SMB.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['SMB'] = list(itertools.chain(*SMB_column))

HML_column = [[HML.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['HML'] = list(itertools.chain(*HML_column))
data = sm.add_constant(data.dropna())

In [12]:
data.head()

const    return       SMB       HML
date       order_book_id                                     
2012-01-04 000004.XSHE      1.0 -0.022250  0.005983  0.005302
           000028.XSHE      1.0 -0.045433  0.005983  0.005302
           000150.XSHE      1.0 -0.030295  0.005983  0.005302
           000153.XSHE      1.0 -0.028053  0.005983  0.005302
           000403.XSHE      1.0  0.000000  0.005983  0.005302

## Factor Exposures ($\beta$)

In [13]:
assets = data.index.levels[1].unique()
Y = [data.xs(asset,level=1)['return'] for asset in assets]
X = [data.xs(asset,level=1)[['SMB','HML','const']] for asset in assets]
reg_results = [regression.linear_model.OLS(y,x).fit().params for y,x in zip(Y,X) if not(x.empty or y.empty)]
indices = [asset for y, x, asset in zip(Y, X, assets) if not(x.empty or y.empty)]
betas = pd.DataFrame(reg_results, index=indices)

In [15]:
betas.head()

,SMB,HML,const
000004.XSHE,0.883906,0.048757,0.002002
000028.XSHE,-0.003029,-0.064295,0.001073
000150.XSHE,0.354122,0.066071,0.002031
000153.XSHE,0.620706,-0.082229,0.001405
000403.XSHE,2.032192,11.457418,-0.017412


## Factor Premium

In [36]:
betas = sm.add_constant(betas.drop('const', axis=1))

R = data['return'].mean(axis=0, level=1)

# Second regression step: estimating the risk premia
risk_free_rate = benchmark_df['return'].mean()

final_results = regression.linear_model.OLS(R - risk_free_rate, betas).fit()

final_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 return   R-squared:                       0.398
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     53.26
Date:                Sat, 05 May 2018   Prob (F-statistic):           1.77e-18
Time:                        21:03:25   Log-Likelihood:                 1012.1
No. Observations:                 164   AIC:                            -2018.
Df Residuals:                     161   BIC:                            -2009.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0017   6.72e-05     24.956      0.000       0.002       0.002
SMB        -7.597e-05      0.000     -0.599      0.550      -0.000       0.000
HML            0.0005   4.81e-05      9.695      0.000       0.000       0.001
==============================================================================
Omnibus:                       39.154   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.545
Skew:                           1.087   Prob(JB):                     8.80e-18
Kurtosis:                       5.601   Cond. No.                         3.92
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fama-Macbeth Test Conclusion: 
although our individual factors are significant, we have a very low  $R^2$ . What this may suggest is that there is a real link between our factors and the returns of our assets, but that there still remains a lot of unexplained noise!